In [47]:
import numpy as np
import torch
from torch.utils.data import Dataset

# Step 1: Generate a distance matrix
n = 50
# distance_matrix = np.random.rand(n, n)
# values between 0 and 100
distance_matrix = np.random.randint(0, 100, (n, n)).astype(float)
# Make the matrix symmetric and zero diagonal
distance_matrix = (distance_matrix + distance_matrix.T) / 2
np.fill_diagonal(distance_matrix, 0)


# Step 2: Generate a dataset of possible routes
class RoutesDataset(Dataset):
    def __init__(self, distance_matrix, num_routes):
        self.distance_matrix = distance_matrix
        self.num_routes = num_routes

    def __len__(self):
        return self.num_routes

    def __getitem__(self, idx):
        route1 = np.random.permutation(len(self.distance_matrix))
        distance1 = sum(self.distance_matrix[route1[i - 1], route1[i]] for i in range(len(route1)))
        
        # normalize route
        route1 = torch.tensor(route1, dtype=torch.float) / (len(route1) - 1)

        return (route1, distance1)


routes_dataset = RoutesDataset(distance_matrix, 1_000)

# Compute variance of the distances
distances = [routes_dataset[i][1] for i in range(len(routes_dataset))]
print(f"Variance of distances: {np.var(distances):.2f}")

Variance of distances: 20019.16


In [48]:
import torch.nn as nn


class RouteRegressor(nn.Module):
    def __init__(self):
        super(RouteRegressor, self).__init__()
        self.fc1 = nn.Linear(n, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, route):
        x = torch.relu(self.fc1(route))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)  # output is a single continuous value


class RouteRegressorCNN(nn.Module):
    def __init__(self):
        super(RouteRegressorCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * n, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, route):
        route = route.unsqueeze(1)  # add an extra dimension for the channels
        x = torch.relu(self.conv1(route))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # flatten the tensor
        x = torch.relu(self.fc1(x))
        return self.fc2(x)


regressor = RouteRegressor()
route1 = torch.randn(32, n)
output = regressor(route1)

In [49]:
from torch.utils.data import DataLoader
from torch.optim import Adam

# Prepare the data
data_loader = DataLoader(routes_dataset, batch_size=32, shuffle=True)

# Initialize the classifier and the optimizer
# regressor = RouteRegressor()
regressor = RouteRegressorCNN()
optimizer = Adam(regressor.parameters(), lr=0.001)

# Train the classifier
for epoch in range(1_000):  # number of epochs

    loss_sum = 0
    correct_predictions = 0
    total_predictions = 0
    for (route, distance) in data_loader:
        # Forward pass
        output = regressor(route.float())
        predictions = output.round()  # compute the predictions
        loss = torch.mean((output - distance.float()) ** 2)  # compute the loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
    if epoch % 100 == 0:
        print(f"Epoch {epoch} - Loss: {loss_sum / len(data_loader):_.2f}")

Epoch 0 - Loss: 5_475_197.78
Epoch 100 - Loss: 19_944.78
Epoch 200 - Loss: 20_576.76
Epoch 300 - Loss: 19_860.32
Epoch 400 - Loss: 19_128.95
Epoch 500 - Loss: 18_915.35
Epoch 600 - Loss: 20_017.52
Epoch 700 - Loss: 19_161.52
Epoch 800 - Loss: 18_314.53
Epoch 900 - Loss: 19_683.40
